# 5章

## 単純なレイヤの実装

### 乗算レイヤの実装

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


$$y = x_3 \times (x_1 \times x_2)$$
$$x_1 = 100, x_2 = 2, x_3 = 1.1$$
$$y = x_3 \times t$$
$$t = x_1 \times x_2$$
$$\frac{\partial y}{\partial t} = x_3$$
$$\frac{\partial t}{\partial x_1} = x_2$$
$$
\frac{\partial y}{\partial x_1} = \frac{\partial y}{\partial t}\frac{\partial t}{\partial x_1}\\
= x_3 \times x_2
$$

とりあえず$x_1$での微分を式にしてみた。
$x_3$での微分を考えてみると、$(x_1 \times x_2)$がその値になる。
順伝播のときに値をキャプチャしておけば、それがそのまま使えるということか。
数式でイメージすると、微分を計算してから実際の値を代入して実際の微分の値を計算するイメージだけど。
いきなり実際の値を当てはめながら計算していけるのか。
なんか騙されたような釈然としない用な感覚だ。

### 加算レイヤの実装

In [6]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# foward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


---

## 活性化関数レイヤの実装

In [9]:
import numpy as np

NumPyの配列が入ってくる前提だけれども、各配列のセルごとに計算をしているだけで他のセルは関係しないからまだ簡単だね。

加算や乗算などは二項演算だったけれどもこれは単項のみだからとてもシンプル。

### ReLUレイヤ

In [8]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

### Sigmoidレイヤ

$$y = \frac{1}{1+\exp(-x)}$$
$$
\frac{\partial y}{\partial x} = -\frac{1}{(1 + \exp(-x))^2} \cdot \exp(-x) \cdot (-1)\\
= \frac{1}{1+\exp(-x)} \cdot \frac{\exp(-x)}{1+\exp(-x)}\\
= \frac{1}{1+\exp(-x)} \cdot (1 - \frac{1}{1+\exp(-x)})\\
= y(1-y)
$$

でかい計算グラフの中ほどにSigmoidがあった場合は、逆伝播で$y(1-y)$をかけ合わせて上流に流せば良い。
$y$はSigmoidノードの出力なので、順伝播のときにSigmoidの出力を記憶しておけば、逆伝播でその出力を使った$y(1-y)$をかけ合わせて上に流せば良い。

In [10]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        seelf.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1.0 - self.out)
        
        return dx

## Affine/Softmaxレイヤの実装

### Affineレイヤ

活性化関数の手前の重み付き加算とバイアスの加算について。
入力のベクトルに重みの行列をかけ合わせたベクトルにさらにバイアスのベクトルを足し合わせる。
それをシグモイドなりなんなりの活性化関数にかける。

バイアスは重みと違って張り巡らされたネットワークではなくその後の一律の加算の部分だね。だから行列で例えば2個のノードから3個のノードへの値を持つ重み行列とは違って、その後の3個に対してバイアスを持てば良いのでバイアスはベクトルになっている。

![memo](memo.jpg)

この図でいうと活性化関数は単項演算に当たるのでパラメータは無いということになるのかな。

活性化関数のelement-wiseな計算とは違って、ここでは行列の積が出てくる。
この行列の内積演算について微分を行わなければならない。

本では計算グラフのノード間を流れる値を行列に拡張して単純なグラフで表している。
上の画像のような細かいノードの結びつきではなくて行列の積をそのまま扱う形で説明しているね。

この部分のグラフについては
$$Y = X \cdot W + B$$
という計算式になる。

$$Y = T + B$$
$$T = X \cdot W$$
ということになるのかな。

$\frac{\partial L}{\partial X}$と$\frac{\partial L}{\partial W}$を求めたい。$L$は全体の計算グラフ。

本によれば
$$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \cdot W^{\mathrm{T}}$$
$$\frac{\partial L}{\partial W} = X^{\mathrm{T}} \cdot \frac{\partial L}{\partial Y}$$
だそうだ。

行列での内積の連鎖律がこうなるということなのかな。
本では細かい解説が省かれているので実際に計算してみるか。

$$Y = f(X \cdot W)$$
とする。

$$Y = f(T)$$
$$T = X \cdot W$$
ということになる。

ここで$f$はelement-wiseな計算ね。
$$f(T) \equiv \left(
    \begin{matrix}
        f(t_{11}) & \ldots & f(t_{n1})\\
        \vdots & \ddots & \vdots\\
        f(t_{1m}) & \ldots & f(t_{nm})
    \end{matrix}
\right)$$

2x2で手計算したらそれらしくなったけれども、なにかおかしい気がする。内積したあとで微分するならば正しくなるけれども、微分したのを内積したらだめじゃね？

連鎖率についてきちんと勉強しないとだめだ。1変数のときは合成関数の微分で簡単だけれども。変数が複数になるとそうは行かないようだ。

学部時代のノートを引っ張り出してきてみてみたけれども、行列を使わずに計算しているので計算式がだいぶ違ってあれだ。

- [誤差逆伝播法をはじめからていねいに - Qiita](https://qiita.com/43x2/items/50b55623c890564f1893)

こちらにこの部分に関する式展開があった。

### バッチ版Affineレイヤ

入力をまとめて行列にしてしまうのだったね。
バイアスの加算にはブロードキャストが行われている。

次はバッチサイズが2個だった場合。
入力が3次のベクトルが2つになって、行列の積が行われる。その結果にバイアスをブロードキャストで足していく。

In [11]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

In [12]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [13]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

バイアスの逆伝播はバッチの数だけ合算したものと考えることになるらしい。

In [14]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [15]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

先に実装した活性化関数はバッチにしてもうまく働くね。なにしろセル一つに対して実行すれば良くて、NumPyでelement-wiseに作られている。

このアフィンレイヤはelement-wiseで処理すれば良い活性化関数とは異なってくるので注意が必要ということに。

In [16]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(selg, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

---

### Softmax-with-Lossレイヤ

In [17]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [18]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [19]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx